#**Budget Planning App for Students (Python + CSV)**

In [ ]:
#Installing required libraries
!pip -q install pandas plotly ipywidgets

In [ ]:
#Importing libraries, folders and setting up paths

import os, csv, math
from datetime import datetime, date
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output

# Base paths
BASE_DIR = "/content/budget_app"
DATA_DIR = os.path.join(BASE_DIR, "data")
REPORTS_DIR = os.path.join(BASE_DIR, "reports")
CSV_PATH = os.path.join(DATA_DIR, "budget_transactions.csv")

COLUMNS = ["Date", "Category", "Type", "Amount"]  # Type: Income or Expense

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# Create CSV with header
if not os.path.exists(CSV_PATH) or os.path.getsize(CSV_PATH) == 0:
    with open(CSV_PATH, "w", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow(COLUMNS)

print("Base dir:", BASE_DIR)
print("CSV path:", CSV_PATH)
print("Reports dir:", REPORTS_DIR)

In [ ]:
#Helper functions

def parse_date_input(raw: str) -> date:
    raw = (raw or "").strip()
    if raw == "":
        return date.today()
    fmts = ["%Y-%m-%d", "%d-%m-%Y", "%d/%m/%Y", "%Y/%m/%d", "%d-%b-%Y", "%b %d %Y"]
    for fmt in fmts:
        try:
            return datetime.strptime(raw, fmt).date()
        except Exception:
            pass
    raise ValueError("Could not parse date. Use YYYY-MM-DD (e.g. 2025-08-16).")

def parse_month_input(raw: str, df: pd.DataFrame = None) -> str:
    """
    Return month string 'YYYY-MM'.
    If raw blank & df provided -> return latest month from df (preferred).
    If raw blank & df empty -> current month.
    """
    raw = (raw or "").strip()
    if raw == "" and df is not None and not df.empty:
        months = sorted(df["Month"].unique())
        if months:
            return months[-1]
    if raw == "":
        t = date.today()
        return f"{t.year}-{t.month:02d}"
    if "-" in raw:
        a,b = raw.split("-")
        if len(a) == 4:
            y, m = int(a), int(b)
        else:
            m, y = int(a), int(b)
    else:
        if len(raw) == 6:
            y, m = int(raw[:4]), int(raw[4:])
        else:
            raise ValueError("Use YYYY-MM (e.g. 2025-08)")
    datetime(y, m, 1)  # validate
    return f"{y}-{m:02d}"

def load_df() -> pd.DataFrame:
    if not os.path.exists(CSV_PATH):
        return pd.DataFrame(columns=COLUMNS)
    df = pd.read_csv(CSV_PATH)
    if df.empty:
        return pd.DataFrame(columns=COLUMNS)
    df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
    df = df.dropna(subset=["Date"])  # drop rows with invalid dates
    df["Category"] = df["Category"].astype(str).str.strip().replace({"": "Uncategorized"})
    df["Type"] = df["Type"].astype(str).str.title().replace({"Income":"Income","Expense":"Expense"})
    df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce").fillna(0.0)
    df["Month"] = df["Date"].dt.to_period("M").astype(str)  # 'YYYY-MM'
    return df

def save_transaction(dt: date, category: str, ttype: str, amount: float) -> None:
    ttype = ttype.title()
    if ttype not in {"Income", "Expense"}:
        raise ValueError("Type must be 'Income' or 'Expense'")
    if amount <= 0 or math.isnan(amount):
        raise ValueError("Amount must be a positive number")
    with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
        csv.writer(f).writerow([dt.strftime("%Y-%m-%d"), category.strip() or "Uncategorized", ttype, f"{amount:.2f}"])

In [ ]:
#Dashboard

def build_dashboard():
    df = load_df()
    if df.empty:
        print("No data yet")
        return

    # Aggregates
    monthly = df.groupby(["Month","Type"])["Amount"].sum().reset_index()
    pivot = monthly.pivot(index="Month",columns="Type",values="Amount").fillna(0)
    pivot["Balance"] = pivot.get("Income",0) - pivot.get("Expense",0)

    cat_data = df[df["Type"]=="Expense"].groupby(["Month","Category"])["Amount"].sum().reset_index()

    months = sorted(df["Month"].unique())
    default_month = months[-1]

    # --- Base subplot layout ---
    fig = make_subplots(
        rows=2, cols=2,
        specs=[[{"type":"xy"},{"type":"domain"}],
               [{"type":"xy"},{"type":"indicator"}]],
        subplot_titles=("Income vs Expense","Expenses by Category","Balance Trend","Savings %")
    )

    # Bar: Income vs Expense
    fig.add_trace(go.Bar(x=pivot.index,y=pivot["Income"],name="Income"),row=1,col=1)
    fig.add_trace(go.Bar(x=pivot.index,y=pivot["Expense"],name="Expense"),row=1,col=1)

    # Pie: default month
    default_cat = cat_data[cat_data["Month"]==default_month]
    fig.add_trace(go.Pie(labels=default_cat["Category"],values=default_cat["Amount"],name="Pie"),row=1,col=2)

    # Line: Balance trend
    fig.add_trace(go.Scatter(x=pivot.index,y=pivot["Balance"],mode="lines+markers",name="Balance"),row=2,col=1)

    # Gauge: Savings %
    inc = df[(df["Month"]==default_month)&(df["Type"]=="Income")]["Amount"].sum()
    exp = df[(df["Month"]==default_month)&(df["Type"]=="Expense")]["Amount"].sum()
    ratio = (max(inc-exp,0)/inc*100) if inc>0 else 0
    fig.add_trace(go.Indicator(mode="gauge+number",value=ratio,
                               gauge={'axis':{'range':[0,100]}},
                               title={'text':f"Savings % ({default_month})"}),row=2,col=2)

    # --- Month dropdown only ---
    month_buttons = []
    for m in months:
        inc = df[(df["Month"]==m)&(df["Type"]=="Income")]["Amount"].sum()
        exp = df[(df["Month"]==m)&(df["Type"]=="Expense")]["Amount"].sum()
        ratio = (max(inc-exp,0)/inc*100) if inc>0 else 0
        cat = cat_data[cat_data["Month"]==m]

        month_buttons.append(dict(
            label=m,
            method="update",
            args=[{"labels":[cat["Category"]],"values":[cat["Amount"]],"value":[ratio]}, # updates pie+gauge
                  {"title":f"Budget Dashboard — {m}"}]
        ))

    fig.update_layout(
        template="plotly_dark",
        barmode="group",
        height=800,width=1100,
        updatemenus=[
            dict(buttons=month_buttons, direction="down", showactive=True, x=0.1, y=1.2,
                 xanchor="left",yanchor="top")
        ],
        annotations=[dict(text="Select Month:",x=0,y=1.25,xref="paper",yref="paper",showarrow=False)]
    )

    fig.show()

# Show fixed dashboard
build_dashboard()

In [ ]:
#Feeding sample data to see the working

demo = [
    ("2025-02-01","Salary","Income",20000),
    ("2025-02-02","Food","Expense",600),
    ("2025-02-05","Transport","Expense",250),
    ("2025-02-12","Shopping","Expense",1200),
    ("2025-03-01","Salary","Income",20000),
    ("2025-03-03","Food","Expense",700),
    ("2025-03-10","Rent","Expense",5000),
    ("2025-03-18","Freelance","Income",4000),
    ("2025-03-21","Travel","Expense",800),
]
for d,c,t,a in demo:
    save_transaction(parse_date_input(d), c, t, a)
print("Demo data added.")



In [ ]:
#Showing transations

def show_recent(n=10):
    df = load_df()
    if df.empty:
        print("No transactions yet.")
        return
    disp = df.sort_values("Date").tail(n).copy()
    disp["Date"] = disp["Date"].dt.strftime("%Y-%m-%d")
    display(disp.reset_index(drop=True))

# Example: show_recent(15)
show_recent(10)

$$......................Thank You......................$$